In [39]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load tf-idf vectors (min_df = 0.3, max_df = 0.7)
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [40]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_df_sub_dir)

In [41]:
# convert to dense matrix (easier computation on K-means)
tfidf_matrix = tfidf_matrix.todense()

In [42]:
tfidf_matrix.shape

(27885L, 488L)

# Graph Clustering
focus on largest connected component on **undirected scotus**

In [43]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

## modularity on undirected scotus
"For a given division of the network's vertices into some modules, modularity reflects the concentration of edges within modules compared with random distribution of links between all nodes regardless of modules"--*Wikipedia*

In [7]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

print mod_clust.summary()

# save clusters in pandas
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 126 clusters
Wall time: 1min 27s


In [10]:
graph_clusters_mod_score = g.modularity(mod_clust.membership)

print graph_clusters_mod_score

0.465097751507


## walktrap on undirected scotus

In [11]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

# save clusters in pandas
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 43s


In [12]:
walktrap_clusters_mod_score = g.modularity(wt_clust.membership)

print walktrap_clusters_mod_score

0.504445233426


# NLP Clustering
K-Means, Gaussian Mixture Models (GMM), Hierarchical Clustering

## K-Means Clustering on tf-idf (tweaked df)

### K-means clustering with K = 10

In [44]:
%%time

# set number of clusters
num_clusters = 10

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_tfidf_clusters = km.labels_.tolist()

Wall time: 34.4 s


cluster_centers_ : array, [n_clusters, n_features]  
Coordinates of cluster centers  
labels_ : :  
Labels of each point  
inertia_ : float  
Sum of distances of samples to their closest cluster center.  

In [51]:
km.cluster_centers_

array([[ 0.01259158,  0.01215159,  0.01057906, ...,  0.01004653,
         0.02102381,  0.01479391],
       [ 0.01621444,  0.01389982,  0.01293384, ...,  0.00941208,
         0.03414376,  0.03318864],
       [ 0.01652844,  0.01613021,  0.01417864, ...,  0.00883425,
         0.02492242,  0.06466142],
       ..., 
       [ 0.02203724,  0.01687924,  0.01518444, ...,  0.00833574,
         0.02372555,  0.22295113],
       [ 0.01300217,  0.00966268,  0.00904081, ...,  0.00868744,
         0.00871948,  0.02099383],
       [ 0.01735869,  0.01563974,  0.01485332, ...,  0.0052997 ,
         0.03141815,  0.05526024]])

In [15]:
nlp_tfidf_clusters = clusters['km_10'].tolist()

In [16]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score = g.modularity(nlp_tfidf_clusters)
print nlp_tfidf_clusters_mod_score

0.000135285942698


### K-means clustering with K = 100

In [10]:
%%time

# set number of clusters
num_clusters = 100

# run kmeans
km2 = KMeans(n_clusters=num_clusters)
km2.fit(tfidf_matrix)

nlp_tfidf_clusters2 = km2.labels_.tolist()

Wall time: 2min 17s


In [17]:
nlp_tfidf_clusters2 = clusters['km_100'].tolist()

In [18]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score2 = g.modularity(nlp_tfidf_clusters2)
print nlp_tfidf_clusters_mod_score2

-2.69555906588e-05


### K-means clustering with K = 1000

In [11]:
%%time

# set number of clusters
num_clusters = 1000

# run kmeans
km3 = KMeans(n_clusters=num_clusters)
km3.fit(tfidf_matrix)

nlp_tfidf_clusters3 = km3.labels_.tolist()

Wall time: 12min 44s


In [19]:
nlp_tfidf_clusters3 = clusters['km_1000'].tolist()

In [20]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score3 = g.modularity(nlp_tfidf_clusters3)
print nlp_tfidf_clusters_mod_score3

-8.97299879181e-05


## GMM Clustering on tf-idf (tweaked df)

### GMM Clustering with K = 10

In [12]:
%%time

# set number of components
num_components = 10

# run GMM
gmm = GaussianMixture(n_components=num_components)
gmm.fit(tfidf_matrix)

gmm_clusters = gmm.predict(tfidf_matrix).tolist()
gmm_clusters = map(int, gmm_clusters)

Wall time: 11min 22s


In [21]:
gmm_clusters = clusters['gmm_10'].tolist()

In [22]:
# modularity score of cluster
gmm_clusters_mod_score = g.modularity(gmm_clusters)
print gmm_clusters_mod_score

4.14573453016e-05


### GMM Clustering with K = 100 

In [13]:
%%time

# set number of components
num_components = 100

# run GMM
gmm2 = GaussianMixture(n_components=num_components)
gmm2.fit(tfidf_matrix)

gmm_clusters2 = gmm2.predict(tfidf_matrix).tolist()
gmm_clusters2 = map(int, gmm_clusters2)

Wall time: 6min 1s


In [23]:
gmm_clusters2 = clusters['gmm_100'].tolist()

In [24]:
# modularity score of cluster
gmm_clusters_mod_score2 = g.modularity(gmm_clusters2)
print gmm_clusters_mod_score2

-0.000376869533807


### GMM Clustering with K = 1000

In [14]:
%%time

# set number of components
num_components = 1000

# run GMM
gmm3 = GaussianMixture(n_components=num_components)
gmm3.fit(tfidf_matrix)

gmm_clusters3 = gmm3.predict(tfidf_matrix).tolist()
gmm_clusters3 = map(int, gmm_clusters3)

Wall time: 46min 2s


In [25]:
gmm_clusters3 = clusters['gmm_1000'].tolist()

In [26]:
# modularity score of cluster
gmm_clusters_mod_score3 = g.modularity(gmm_clusters3)
print gmm_clusters_mod_score3

-0.000128866437982


## Hierarchical Clustering on tf-idf (tweaked df)

### Hierarchical Clustering with K = 10

In [4]:
%%time

# set number of components
num_clusters = 10

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters = hc.fit_predict(tfidf_matrix).tolist()
hc_clusters = map(int, hc_clusters)

Wall time: 4min 42s


In [27]:
hc_clusters = clusters['hc_10'].tolist()

In [28]:
# modularity score of cluster
hc_clusters_mod_score = g.modularity(hc_clusters)
print hc_clusters_mod_score

-0.000733265904717


### Hierarchical Clustering with K = 100

In [5]:
%%time

# set number of components
num_clusters = 100

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc2 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters2 = hc2.fit_predict(tfidf_matrix).tolist()
hc_clusters2 = map(int, hc_clusters2)

Wall time: 4min 22s


In [29]:
hc_clusters2 = clusters['hc_100'].tolist()

In [30]:
# modularity score of cluster
hc_clusters_mod_score2 = g.modularity(hc_clusters2)
print hc_clusters_mod_score2

-0.000342074663494


### Hierarchical Clustering with K = 1000

In [6]:
%%time

# set number of components
num_clusters = 1000

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc3 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters3 = hc3.fit_predict(tfidf_matrix).tolist()
hc_clusters3 = map(int, hc_clusters3)

Wall time: 4min 21s


In [31]:
hc_clusters3 = clusters['hc_1000'].tolist()

In [32]:
# modularity score of cluster
hc_clusters_mod_score3 = g.modularity(hc_clusters3)
print hc_clusters_mod_score3

-4.8856955687e-05


## Compare NLP clustering (tfidf) vs graph clustering

In [15]:
#clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])
clusters = pd.DataFrame(index=op_id_to_bow_id, columns=[])

# add in communities 
clusters['mod'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['mod'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['mod'] = clusters['mod'].astype(np.int)

# add in walktrap clusters
clusters['wt'] = walktrap_clusters
clusters['wt'].fillna(max(walktrap_clusters) + 1, inplace=True)
clusters['wt'] = clusters['wt'].astype(np.int)

# add in NLP clusters
clusters['km_10'] = nlp_tfidf_clusters
clusters['km_100'] = nlp_tfidf_clusters2
clusters['km_1000'] = nlp_tfidf_clusters3

clusters['gmm_10'] = gmm_clusters
clusters['gmm_100'] = gmm_clusters2
clusters['gmm_1000'] = gmm_clusters3

clusters['hc_10'] = hc_clusters
clusters['hc_100'] = hc_clusters2
clusters['hc_1000'] = hc_clusters3

clusters.to_csv(csv_dir + "clusters_tweak_df.csv")

### load cluster csv saved in current directory

In [52]:
clusters = pd.read_csv(csv_dir + 'clusters_tweak_df.csv')
clusters

,Unnamed: 0,mod,wt,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
0,145658,1,5,4,0,927,3,3,966,7,38,43
1,89370,3,294,4,38,413,3,99,247,7,38,143
2,89371,0,35,7,92,129,7,68,646,9,63,680
3,89372,0,3,0,16,271,5,88,491,7,38,483
4,89373,0,3,6,93,722,1,24,369,3,80,530
5,89374,2,4,6,23,149,1,77,75,5,58,454
6,89375,2,5,2,13,988,5,88,793,7,38,581
7,89376,2,6,2,37,136,5,72,995,7,38,344
8,89377,2,7,1,48,196,3,74,262,1,96,198
9,89378,2,7,2,21,632,5,75,444,7,12,642


# MAJOR CORRECTION (modularity and walktrap have different indices than NLP clusters!!!)
make separate csv for mod and wt column  
delete these two columns from original csv

In [53]:
clusters = clusters.set_index('Unnamed: 0')
clusters.index.name=None
clusters = clusters[["km_10","km_100","km_1000","gmm_10","gmm_100","gmm_1000","hc_10","hc_100","hc_1000"]]
clusters.to_csv(csv_dir + 'clusters_tweak_df.csv')

In [54]:
clusters

,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
145658,4,0,927,3,3,966,7,38,43
89370,4,38,413,3,99,247,7,38,143
89371,7,92,129,7,68,646,9,63,680
89372,0,16,271,5,88,491,7,38,483
89373,6,93,722,1,24,369,3,80,530
89374,6,23,149,1,77,75,5,58,454
89375,2,13,988,5,88,793,7,38,581
89376,2,37,136,5,72,995,7,38,344
89377,1,48,196,3,74,262,1,96,198
89378,2,21,632,5,75,444,7,12,642


# Modularity Scores of Clusters

In [33]:
clusters_strings = ['mod', 'wt', 'km_10', 'km_100', 'km_1000', 'gmm_10', 'gmm_100', 'gmm_1000', 'hc_10', 'hc_100', 'hc_1000']
mod_scores = [graph_clusters_mod_score, walktrap_clusters_mod_score, nlp_tfidf_clusters_mod_score, nlp_tfidf_clusters_mod_score2,
             nlp_tfidf_clusters_mod_score3, gmm_clusters_mod_score, gmm_clusters_mod_score2, gmm_clusters_mod_score3,
             hc_clusters_mod_score, hc_clusters_mod_score2, hc_clusters_mod_score3]

clusters = pd.DataFrame(index=clusters_strings)
clusters['modularity_score'] = mod_scores

clusters.to_csv(csv_dir + "clusters_tweak_df_mod_score.csv")

In [34]:
clusters_mod_score = pd.read_csv(csv_dir + 'clusters_tweak_df_mod_score.csv')
clusters_mod_score

,Unnamed: 0,modularity_score
0,mod,0.465098
1,wt,0.504445
2,km_10,0.000135
3,km_100,-0.000027
4,km_1000,-0.000090
5,gmm_10,0.000041
6,gmm_100,-0.000377
7,gmm_1000,-0.000129
8,hc_10,-0.000733
9,hc_100,-0.000342


# normalized_mutual_information score (nmi) of 
## 1. modularity vs. walktrap
## 2. every combination of nlp_clusters
note: can't do nmi(graph_cluster, nlp_cluster) because len(graph_cluster) != len(nlp_cluster)  
http://scikit-learn.org/stable/modules/classes.html#clustering-metrics

In [37]:
string_list = ["km_10", "km_100", "km_1000", "gmm_10", "gmm_100", "gmm_1000", 
               "hc_10", "hc_100", "hc_1000"]
clusters_list = [nlp_tfidf_clusters, nlp_tfidf_clusters2, nlp_tfidf_clusters3,
                gmm_clusters, gmm_clusters2, gmm_clusters3, hc_clusters, hc_clusters2, hc_clusters3]

nums = range(0,9)
combs = list(combinations(nums,2))

'''
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm
'''

combinations_list = []
nmi_scores = []
ami_scores = []
mi_scores = []
ar_scores = []
#ch_scores = []
cs_scores = []
fm_scores = []
#hcvm_scores = []
hs_scores = []
#ss_scores = []
#ss2_scores = []
vm_scores = []

combinations_list.append("mod vs. wt")
nmi_scores.append(nmi(graph_clusters, walktrap_clusters))
ami_scores.append(ami(graph_clusters, walktrap_clusters))
mi_scores.append(mi(graph_clusters, walktrap_clusters))
ar_scores.append(ar(graph_clusters, walktrap_clusters))
#ch_scores.append(ch(graph_clusters, walktrap_clusters))
cs_scores.append(cs(graph_clusters, walktrap_clusters))
fm_scores.append(fm(graph_clusters, walktrap_clusters))
#hcvm_scores.append(hcvm(graph_clusters, walktrap_clusters))
hs_scores.append(hs(graph_clusters, walktrap_clusters))
#ss_scores.append(ss(graph_clusters, walktrap_clusters))
#ss2_scores.append(ss2(graph_clusters, walktrap_clusters))
vm_scores.append(vm(graph_clusters, walktrap_clusters))

for i in combs:
    combinations_list.append(string_list[i[0]] + " vs. " + string_list[i[1]])
    nmi_scores.append(nmi(clusters_list[i[0]], clusters_list[i[1]]))
    ami_scores.append(ami(clusters_list[i[0]], clusters_list[i[1]]))
    mi_scores.append(mi(clusters_list[i[0]], clusters_list[i[1]]))
    ar_scores.append(ar(clusters_list[i[0]], clusters_list[i[1]]))
    #ch_scores.append(ch(clusters_list[i[0]], clusters_list[i[1]]))
    cs_scores.append(cs(clusters_list[i[0]], clusters_list[i[1]]))
    fm_scores.append(fm(clusters_list[i[0]], clusters_list[i[1]]))
    #hcvm_scores.append(hcvm(clusters_list[i[0]], clusters_list[i[1]]))
    hs_scores.append(hs(clusters_list[i[0]], clusters_list[i[1]]))
    #ss_scores.append(ss(clusters_list[i[0]], clusters_list[i[1]]))
    #ss2_scores.append(ss2(clusters_list[i[0]], clusters_list[i[1]]))
    vm_scores.append(vm(clusters_list[i[0]], clusters_list[i[1]]))
    
clusters = pd.DataFrame(index=combinations_list)
clusters["nmi_score"] = nmi_scores
clusters["ami_score"] = ami_scores
clusters["mi_score"] = mi_scores
clusters["ar_score"] = ar_scores
clusters["cs_score"] = cs_scores
clusters["fm_score"] = fm_scores
clusters["hs_score"] = hs_scores
clusters["vm_score"] = vm_scores
clusters.to_csv(csv_dir + "clusters_tweak_df_nmi_score.csv")

In [38]:
clusters_nmi_score = pd.read_csv(csv_dir + 'clusters_tweak_df_nmi_score.csv')
clusters_nmi_score

,Unnamed: 0,nmi_score,ami_score,mi_score,ar_score,cs_score,fm_score,hs_score,vm_score
0,mod vs. wt,0.384960,0.184582,1.001603,0.088432,0.222527,0.233443,0.665962,0.333588
1,km_10 vs. km_100,0.448432,0.310733,1.415778,0.097096,0.313196,0.205423,0.642063,0.421020
2,km_10 vs. km_1000,0.421746,0.220471,1.626462,0.013223,0.241144,0.076337,0.737609,0.363462
3,km_10 vs. gmm_10,0.696169,0.691928,1.544045,0.649731,0.692129,0.691210,0.700233,0.696157
4,km_10 vs. gmm_100,0.444191,0.307625,1.403013,0.099450,0.310096,0.209347,0.636274,0.416974
5,km_10 vs. gmm_1000,0.422754,0.221029,1.630625,0.012939,0.241679,0.075207,0.739497,0.364300
6,km_10 vs. hc_10,0.368210,0.365001,0.805766,0.249611,0.371021,0.341563,0.365419,0.368199
7,km_10 vs. hc_100,0.375391,0.261728,1.175163,0.091939,0.264416,0.189538,0.532943,0.353464
8,km_10 vs. hc_1000,0.408956,0.212315,1.580564,0.012047,0.233324,0.071619,0.716794,0.352051
9,km_100 vs. km_1000,0.646280,0.417299,3.568569,0.119798,0.529086,0.201169,0.789433,0.633556
